In [ ]:
# 🧠 EyePop.ai - Person Detection Workshop
# To run this notebook, you must provide your API credentials below

# ✅ Step 1: Install required libraries
!pip install -q eyepop pillow matplotlib python-dotenv requests

# ✅ Step 2: Provide your EyePop credentials

In [ ]:
# flake8: noqa

# --- Setup and Imports ---
# EyePop SDK for computer vision
from eyepop import EyePopSdk

# Image display and processing
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from io import BytesIO

# Networking and formatting
import requests
import json

# --- Credentials & Image URL ---
POP_ID = "<Add Your Pop UUID>"
SECRET_KEY = "<Add Your API Key>"
example_image_url = 'https://github.com/eyepop-ai/Labs/blob/main/python/PersonDetectionWorkshop/images/person4.jpg?raw=true'

# --- Load and Predict ---
with EyePopSdk.workerEndpoint(pop_id=POP_ID, secret_key=SECRET_KEY) as endpoint:
    result = endpoint.load_from(example_image_url).predict()
    print("Full Prediction Result:")
    print(json.dumps(result, indent=4))

    # Filter predictions with confidence > 0.80
    filtered_objects = [obj for obj in result["objects"] if obj["confidence"] > 0.80]
    filtered_result = {**result, "objects": filtered_objects}

# --- Display Image with Annotations ---
def draw_pretty_keypoints(ax, objects):
    for obj in objects:
        if obj["classLabel"] != "person":
            continue

        keypoints = obj["keyPoints"][0]["points"]
        for point in keypoints:
            x, y = point["x"], point["y"]
            label = point["classLabel"]

            # Draw a circle marker
            circle = patches.Circle((x, y), radius=4, color="cyan", ec="black", linewidth=1.5, zorder=3)
            ax.add_patch(circle)

            # Add a text label with background
            ax.text(
                x + 6, y - 6, label,
                fontsize=9,
                color="white",
                bbox=dict(boxstyle="round,pad=0.2", fc="black", ec="none", alpha=0.7),
                zorder=4,
            )


response = requests.get(example_image_url)
image = Image.open(BytesIO(response.content))

plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis("off")

# Plot predictions
plot = EyePopSdk.plot(plt.gca())
plot.prediction(filtered_result)

# Optionally, highlight keypoints for 'person' class
draw_pretty_keypoints(plt.gca(), filtered_result["objects"])

plt.title("Detected Persons with Keypoints", fontsize=16)
plt.show()